# Transfer multiple participants

This script allows you to loop through a list of participants.

You'll probably want to use the test notebook (`Flywheel_Data_Transfer.ipynb`) first for one participants to create the credential config file and figure out the correct variable names.


In [1]:
import flywheel
import tarfile
import os
import time
import zipfile
from zipfile import ZipFile
import configparser

## Define functions

In [2]:
def transfer_data (sub, in_prefix, out_prefix, in_project, out_project):
    ipID = in_prefix + sub
    opID = out_prefix + sub
    outpath = '/fmriDataRaw/fmri_data_raw/{PROJECT}'.format(PROJECT=out_project)
    
    if os.path.exists(os.path.join(outpath, opID)) == False:
        print('-------------- Transferring: {} -------------'.format(ipID))
        
        # download tar file
        session = fw.lookup('falklab/{proj}/{pid}'.format(proj=in_project,pid=ipID))
        print ('Downloading tar')
        fw.download_tar(session,'./working_data/{opID}.tar'.format(opID=opID))
        
        # open tar file
        with open("working_data/{opID}.tar".format(opID=opID), 'rb') as f# <--- Flywheel download as Read Bytes
        print ('Opening tar in memory as:',f,'\n')
        tar_data = tarfile.open(fileobj=f, mode='r:') # <--- Unpack tar in memory
        
        # Create the directory if not exist
        output_dicom_dir = '{outpath}/{opID}/'.format(outpath=outpath,opID=opID)

        if not os.path.exists(os.path.dirname(output_dicom_dir)):
            try:
                print('makedirs --> {}'.format(output_dicom_dir))
                os.makedirs(os.path.dirname(output_dicom_dir))
            except:
                print('oops! failed to create --> {}'.format(output_dicom_dir))     

                
        # Extract dicom files
        for member in tar_data.getmembers():
            if 'dicom.zip' in member.name:       # <--- Only extract files with 'dicom.zip' 
                print('Extracting: {}\n'.format(member.name))
        
                tfile = tar_data.extractfile(member.name)
                dicom_zip = zipfile.ZipFile(tfile, mode='r')
                dicom_zip.extractall(output_dicom_dir)
                
        print('Participant {} Complete\n'.format(sub))
        tar_data.close()
    else:
        print('-------------- Skipping existing participant: {} -------------'.format(ipID))
                
                

## Define variables
* home_dir = home directory where the config file was created in the test notebook
* in_project = project name on flywheel
* out_project = desired output project name
* subs = participant numbers minus the prefix (e.g. use 01 for bpp01)
* in_prefix = project prefix listed on flywheel
* out_prefix = desired project prefix

In [3]:
# home_dir = "/home/dcosme@asc.upenn.edu"
# in_project = 'bbprime'
# out_project = "bbprime"
# subs = ['21', '22', '23']
# in_prefix = 'bpp'
# out_prefix = 'BPP'

home_dir = "/home/YOUR_JANUS_USERNAME@asc.upenn.edu"
in_project = ''
out_project = ""
subs = []
in_prefix = ''
out_prefix = ''

## Setup API

In [4]:
# add UPenn Flywheel api key to your config.ini
fw_cred = {}
config = configparser.ConfigParser()

config.read(home_dir + '/configs/config.ini') # <--- add your Twitter API tokens to this file!
for item,value in config['UPENN-FLYWHEEL'].items():
    fw_cred[item]=value

# read your API key
api = fw_cred['apikey']

# Create client using your API key
fw = flywheel.Client(api)

# print your flywheel information & confirm it works as expected
self = fw.get_current_user()
print('UPenn Flywheel User: %s %s (%s)' % 
      (self.firstname, self.lastname, self.email))

UPenn Flywheel User: Dani Cosme (dcosme@upenn.edu)


## Loop through data

In [22]:
for sub in subs:
    transfer_data(sub, in_prefix, out_prefix, in_project, out_project)

-------------- Transferring: BPP25 -------------
Opening tar in memory as: <_io.BufferedReader name='working_data/BPP25.tar'> 

makedirs --> /fmriDataRaw/fmri_data_raw/bbprime/BPP25/
Extracting: scitran/falklab/bbprime/BPP25/CAMRIS^Falk/localizer_multislice/1.3.12.2.1107.5.2.43.66044.2021082712450847392007932.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP25/CAMRIS^Falk/AAHead_Scout/1.3.12.2.1107.5.2.43.66044.202108271246126597808663.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP25/CAMRIS^Falk/AAHead_Scout_MPR_sag/1.3.12.2.1107.5.2.43.66044.2021082712461616259909783.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP25/CAMRIS^Falk/AAHead_Scout_MPR_cor/1.3.12.2.1107.5.2.43.66044.2021082712461616299509789.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP25/CAMRIS^Falk/AAHead_Scout_MPR_tra/1.3.12.2.1107.5.2.43.66044.2021082712461616321509793.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP25/CAMRIS^Falk/PhoenixZIPReport/1.3.12.2.1107.5.2.43.66044.300

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/MPRAGE_TI1100_ipat2/1.3.12.2.1107.5.2.43.66044.2021082511184989437288851.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/task-read_run-1_bold/1.3.12.2.1107.5.2.43.66044.2021082511241026273590275.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/task-read_run-1_epi/1.3.12.2.1107.5.2.43.66044.2021082511323110253879783.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/task-share_run-1_bold/1.3.12.2.1107.5.2.43.66044.2021082511324159558980191.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/task-share_run-1_epi/1.3.12.2.1107.5.2.43.66044.202108251142094821174209.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/task-share_run-2_bold/1.3.12.2.1107.5.2.43.66044.2021082511421954592474617.0.0.0.dicom.zip

Extracting: scitran/falklab/bbprime/BPP42/CAMRIS^Falk/task-share_run-2_epi/1.3.12.2.1107.5.2.43.66044.2021082511510662112268635.0.0